In [1]:
import os
import json
import time
import numpy as np
import pandas as pd
import cv2
import torch
from torch import nn, optim
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Lambda

In [2]:
with open('full_input_original_human_labels.json', 'r') as file:
    human_dict = json.load(file)
    
with open('full_input_original_model_labels.json', 'r') as file:
    model_dict = json.load(file)
    
spot_keys = list(human_dict[list(human_dict.keys())[0]].keys())

In [3]:
def build_df(method_keys):
    totals = {}
    errors = {}
    false_positives = {}
    false_negatives = {}
    for spot_key in spot_keys:
        totals[spot_key] = 0
        errors[spot_key] = 0
        false_positives[spot_key] = 0
        false_negatives[spot_key] = 0

    for key in method_keys:
        for spot_key in spot_keys:
            if(human_dict[key][spot_key]):
                totals[spot_key] = totals[spot_key] + 1
            if(human_dict[key][spot_key] != model_dict[key][spot_key]):
                errors[spot_key] = errors[spot_key] + 1
                if(not human_dict[key][spot_key]):
                    false_positives[spot_key] = false_positives[spot_key] + 1
                else:
                    false_negatives[spot_key] = false_negatives[spot_key] + 1

    # Create a structured numpy array
    dtype = [('Times Occupied', int), ('Errors', int), ('False Positives', int), ('False Negatives', int)]
    values = [(totals[key], errors[key], false_positives[key], false_negatives[key]) for key in totals.keys()]
    structured_np = np.array(values, dtype=dtype)
    df = pd.DataFrame(structured_np)

    # Calculate True Negatives
    df["True Negatives"] = len(method_keys) - df["False Negatives"]

    # Calculate Accuracy
    df["Accuracy"] = (df["Times Occupied"] + df["True Negatives"]) / (df["Times Occupied"] + df["True Negatives"] + df["False Positives"] + df["False Negatives"])

    # Calculate Precision
    df["Precision"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Positives"])

    # Calculate Recall
    df["Recall"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Negatives"])

    # Calculate F1 Score
    df["F1 Score"] = 2 * (df["Precision"] * df["Recall"]) / (df["Precision"] + df["Recall"])
    df.index = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6']
    df = df.round(2)
    return df

In [4]:
def build_df_dict(method_keys, meth_dict):
    totals = {}
    errors = {}
    false_positives = {}
    false_negatives = {}
    for spot_key in spot_keys:
        totals[spot_key] = 0
        errors[spot_key] = 0
        false_positives[spot_key] = 0
        false_negatives[spot_key] = 0

    for key in method_keys:
        for spot_key in spot_keys:
            if(human_dict[key][spot_key]):
                totals[spot_key] = totals[spot_key] + 1
            if(human_dict[key][spot_key] != meth_dict[key][spot_key]):
                errors[spot_key] = errors[spot_key] + 1
                if(not human_dict[key][spot_key]):
                    false_positives[spot_key] = false_positives[spot_key] + 1
                else:
                    false_negatives[spot_key] = false_negatives[spot_key] + 1

    # Create a structured numpy array
    dtype = [('Times Occupied', int), ('Errors', int), ('False Positives', int), ('False Negatives', int)]
    values = [(totals[key], errors[key], false_positives[key], false_negatives[key]) for key in totals.keys()]
    structured_np = np.array(values, dtype=dtype)
    df = pd.DataFrame(structured_np)

    # Calculate True Negatives
    df["True Negatives"] = len(method_keys) - df["False Negatives"]

    # Calculate Accuracy
    df["Accuracy"] = (df["Times Occupied"] + df["True Negatives"]) / (df["Times Occupied"] + df["True Negatives"] + df["False Positives"] + df["False Negatives"])

    # Calculate Precision
    df["Precision"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Positives"])

    # Calculate Recall
    df["Recall"] = df["Times Occupied"] / (df["Times Occupied"] + df["False Negatives"])

    # Calculate F1 Score
    df["F1 Score"] = 2 * (df["Precision"] * df["Recall"]) / (df["Precision"] + df["Recall"])
    df.index = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6']
    df = df.round(2)
    return df

In [5]:
interval = 3
count = 0
total = 0
ordered_keys = list(human_dict.keys())
ordered_keys.sort()
training_keys = []
testing_keys = []
# for key in ordered_keys:
#     print(str(count) + ' ' + key[12:24], end='')
#     if(human_dict[key][spot_key] != model_dict[key][spot_key]):
#         print('-xxxxxx', end='')
#     print('',end='\n')
#     count = count + 1

# Model doesn't make a mistake for 
training_keys = ordered_keys[0:171] 
testing_keys = ordered_keys[171:]
validation_keys = ordered_keys[281:]

In [6]:
build_df(training_keys)

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,33,1,1,0,171,1.00,0.97,1.00,0.99
A2,37,0,0,0,171,1.00,1.00,1.00,1.00
A3,28,1,0,1,170,0.99,1.00,0.97,0.98
B1,22,0,0,0,171,1.00,1.00,1.00,1.00
B2,27,0,0,0,171,1.00,1.00,1.00,1.00
B3,69,2,2,0,171,0.99,0.97,1.00,0.99
B4,71,0,0,0,171,1.00,1.00,1.00,1.00
B5,119,1,1,0,171,1.00,0.99,1.00,1.00
B6,89,0,0,0,171,1.00,1.00,1.00,1.00


In [7]:
build_df(testing_keys[:-150])

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,24,6,6,0,60,0.93,0.80,1.00,0.89
A2,26,11,11,0,60,0.89,0.70,1.00,0.83
A3,23,1,0,1,59,0.99,1.00,0.96,0.98
B1,20,3,1,2,58,0.96,0.95,0.91,0.93
B2,18,19,19,0,60,0.80,0.49,1.00,0.65
B3,29,15,15,0,60,0.86,0.66,1.00,0.79
B4,19,4,4,0,60,0.95,0.83,1.00,0.90
B5,29,15,15,0,60,0.86,0.66,1.00,0.79
B6,13,5,5,0,60,0.94,0.72,1.00,0.84


In [8]:
print(len(training_keys))
print(len(testing_keys))

171
210


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layer 1
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)  
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        # Convolutional layer 2
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
        
        # Convolutional layer 3
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.relu3 = nn.ReLU()

        # Convolutional layer 4
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.relu4 = nn.ReLU()

        # Convolutional layer 5
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        # Max pool layer
        self.pool = nn.MaxPool2d(kernel_size=2)

        # Dropout layer
        self.dropout = nn.Dropout(p=0.5)

        # Fully connected layers
        self.fc1 = nn.Linear(512 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 2)

    def forward(self, x):
        # Convolutional layer 1
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.pool(out)

        # Convolutional layer 2
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.pool(out)

        # Convolutional layer 3
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.pool(out)

        # Convolutional layer 4
        out = self.conv4(out)
        out = self.bn4(out)
        out = self.relu4(out)
        out = self.pool(out)

        # Convolutional layer 5
        out = self.conv5(out)
        out = self.bn5(out)
        out = self.relu5(out)
        out = self.pool(out)

        # Flatten for fully connected layer
        out = out.view(out.size(0), -1)

        # Fully connected layer 1
        out = self.fc1(out)
        out = self.dropout(out)

        # Fully connected layer 2
        out = self.fc2(out)
        out = self.dropout(out)

        # Fully connected layer 3
        out = self.fc3(out)

        return out


# Originally in Model_Maker notebook, this preps cropped parking spaces for ML processing
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to 256x256
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize((0.5,0.5,0.5,), (0.5,0.5,0.5,))  # Normalize pixel values in the range [-1, 1]
])

In [10]:
def get_models():
    old_models = {}
    for key in spot_keys:
        model_path = os.path.join('Archive', 'old_models', key + '.pth')
        old_models[key] = CNN()
        old_models[key].load_state_dict(torch.load(model_path)) 
        old_models[key].eval() 
    return old_models

def predict_image(to_predict_image, predicting_model_dict):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ret = {}
    for key in spot_keys:
        dir_name = 'input'
        full_path = os.path.join(dir_name, to_predict_image)
        image = Image.open(full_path)
        spots_file_path = os.path.join('Archive', 'old_models', 'spots.json')
        with open(spots_file_path, 'r') as spots_file:
            spots_data = json.load(spots_file)
        x, x_w, y, y_h = spots_data[key]
        cropped_image = image.crop((x, y, x_w, y_h))
        input_tensor = transform(cropped_image)
        input_tensor = input_tensor.unsqueeze(0)
        
        # Move the input tensor to the same device as the model
        input_tensor = input_tensor.to(device)
        
        with torch.no_grad():
            output = predicting_model_dict[key](input_tensor)
            _, predicted = torch.max(output, 1)

        prediction = predicted.item()
        ret[key] = False
        if prediction == 0: ret[key] = True
    return ret

def bulk_predict(keys, predicting_model_dict):
    ret = {}
    for x in keys:
        ret[x] = predict_image(x, predicting_model_dict)
    return ret

In [11]:
class LotImageDataset(Dataset):
    def __init__(self, img_dict, label_dict, transform=None):
        self.img_dict = img_dict
        self.label_dict = label_dict
        self.transform = transform

    def __len__(self):
        return len(self.img_dict)

    def __getitem__(self, idx):
        img_name = list(self.img_dict.keys())[idx]
        image = self.img_dict[img_name]
        label = self.label_dict[img_name]
        if self.transform:
            image = self.transform(image)
        return image, label

In [12]:
def get_dataset_params(my_dict, selected_keys, spot, spot_locs):
    labels_dict = {}
    images_dict = {}
    for to_predict_image in selected_keys:
        dir_name = 'input'
        image_path = os.path.join(dir_name, to_predict_image)
        to_crop = Image.open(image_path)
        x, x_w, y, y_h = spot_locs[spot]
        images_dict[to_predict_image] = to_crop.crop((x, y, x_w, y_h))
        pred = 1
        if my_dict[to_predict_image][spot]: pred = 0
        labels_dict[to_predict_image] = pred
    return images_dict, labels_dict

In [13]:
spots_file_path = os.path.join('Archive', 'old_models', 'spots.json')
with open(spots_file_path, 'r') as spots_file:
    spots_data = json.load(spots_file)



In [14]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


In [22]:
start_time = time.time()  # Save the current time

# Dictionary to store the models
models_es_dict = {}

# Define patience and best loss for early stopping
patience = 5

# Prepare the model for each parking space
for parking_space in spot_keys:
    images, labels = get_dataset_params(human_dict, training_keys, parking_space, spots_data)
    dataset = LotImageDataset(images, labels, transform=transform)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    images, labels = get_dataset_params(human_dict, validation_keys, parking_space, spots_data)
    dataset_validation = LotImageDataset(images, labels, transform=transform)
    dataloader_validation = DataLoader(dataset, batch_size=16, shuffle=True)
    
    # Use the previously defined CNN class
    model = CNN()
    print(f'{parking_space}: Success!!')

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0000025)

    # Define the learning rate scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # Number of epochs to train for
    num_epochs = 30

    # Move model to GPU if available
    if torch.cuda.is_available():
        model = model.cuda()

    # Initialize best loss for early stopping
    best_loss = np.inf
    no_improve_epochs = 0

    # Train the model
    for epoch in range(num_epochs):
        model.train()  # set the model to training mode
        train_loss = 0.0
        for i, (inputs, labels) in enumerate(dataloader):
            # Move data and labels to GPU if available
            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            # Clear the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        train_loss = train_loss / len(dataloader.dataset)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(dataloader_validation):
                # Move data and labels to GPU if available
                if torch.cuda.is_available():
                    inputs = inputs.cuda()
                    labels = labels.cuda()

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

        val_loss = val_loss / len(dataloader_validation.dataset)

        print('Parking Space: {}, Epoch {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(
            parking_space, epoch + 1, train_loss, val_loss))

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            no_improve_epochs = 0
            best_model_state = model.state_dict()  # Save the model state
        else:
            no_improve_epochs += 1

        if no_improve_epochs == patience:
            print('Early stopping due to no improvement after {} epochs.'.format(patience))
            break

        # Step the scheduler
        scheduler.step()

    print('Finished Training for parking space ', parking_space)

    # Save the model after training
    torch.save(best_model_state, f"{parking_space}.pth")

    # Load the best model state and add the model to the dictionary
    model.load_state_dict(best_model_state)
    models_es_dict[parking_space] = model
    
end_time = time.time()  # Save the current time again after the code block has executed
print("Time elapsed: ", ((end_time - start_time)/60), "minutes")

A1: Success!!
Parking Space: A1, Epoch 1, Training Loss: 0.4390, Validation Loss: 0.5534
Parking Space: A1, Epoch 2, Training Loss: 0.1974, Validation Loss: 0.4892
Parking Space: A1, Epoch 3, Training Loss: 0.1148, Validation Loss: 0.3490
Parking Space: A1, Epoch 4, Training Loss: 0.0725, Validation Loss: 0.1320
Parking Space: A1, Epoch 5, Training Loss: 0.0682, Validation Loss: 0.0498
Parking Space: A1, Epoch 6, Training Loss: 0.0443, Validation Loss: 0.0340
Parking Space: A1, Epoch 7, Training Loss: 0.0284, Validation Loss: 0.0246
Parking Space: A1, Epoch 8, Training Loss: 0.0365, Validation Loss: 0.0186
Parking Space: A1, Epoch 9, Training Loss: 0.0234, Validation Loss: 0.0169
Parking Space: A1, Epoch 10, Training Loss: 0.0251, Validation Loss: 0.0144
Parking Space: A1, Epoch 11, Training Loss: 0.0181, Validation Loss: 0.0146
Parking Space: A1, Epoch 12, Training Loss: 0.0187, Validation Loss: 0.0143
Parking Space: A1, Epoch 13, Training Loss: 0.0269, Validation Loss: 0.0139
Parking

Parking Space: B1, Epoch 20, Training Loss: 0.0168, Validation Loss: 0.0134
Parking Space: B1, Epoch 21, Training Loss: 0.0153, Validation Loss: 0.0140
Parking Space: B1, Epoch 22, Training Loss: 0.0194, Validation Loss: 0.0133
Parking Space: B1, Epoch 23, Training Loss: 0.0140, Validation Loss: 0.0133
Parking Space: B1, Epoch 24, Training Loss: 0.0206, Validation Loss: 0.0127
Parking Space: B1, Epoch 25, Training Loss: 0.0140, Validation Loss: 0.0131
Parking Space: B1, Epoch 26, Training Loss: 0.0164, Validation Loss: 0.0134
Parking Space: B1, Epoch 27, Training Loss: 0.0151, Validation Loss: 0.0136
Parking Space: B1, Epoch 28, Training Loss: 0.0171, Validation Loss: 0.0129
Parking Space: B1, Epoch 29, Training Loss: 0.0163, Validation Loss: 0.0130
Early stopping due to no improvement after 5 epochs.
Finished Training for parking space  B1
B2: Success!!
Parking Space: B2, Epoch 1, Training Loss: 0.5095, Validation Loss: 0.5806
Parking Space: B2, Epoch 2, Training Loss: 0.2524, Validat

Parking Space: B5, Epoch 19, Training Loss: 0.0468, Validation Loss: 0.0394
Parking Space: B5, Epoch 20, Training Loss: 0.0470, Validation Loss: 0.0401
Parking Space: B5, Epoch 21, Training Loss: 0.0443, Validation Loss: 0.0393
Parking Space: B5, Epoch 22, Training Loss: 0.0482, Validation Loss: 0.0393
Parking Space: B5, Epoch 23, Training Loss: 0.0422, Validation Loss: 0.0384
Parking Space: B5, Epoch 24, Training Loss: 0.0502, Validation Loss: 0.0386
Parking Space: B5, Epoch 25, Training Loss: 0.0452, Validation Loss: 0.0390
Parking Space: B5, Epoch 26, Training Loss: 0.0437, Validation Loss: 0.0384
Parking Space: B5, Epoch 27, Training Loss: 0.0504, Validation Loss: 0.0388
Parking Space: B5, Epoch 28, Training Loss: 0.0435, Validation Loss: 0.0390
Parking Space: B5, Epoch 29, Training Loss: 0.0503, Validation Loss: 0.0383
Parking Space: B5, Epoch 30, Training Loss: 0.0487, Validation Loss: 0.0384
Finished Training for parking space  B5
B6: Success!!
Parking Space: B6, Epoch 1, Traini

In [23]:
for model_key in models_es_dict.keys():
    models_es_dict[model_key].eval() 

fresh_model_results = bulk_predict(testing_keys, models_es_dict)
results = build_df_dict(testing_keys, fresh_model_results)
django_results = build_df(testing_keys)
print(f"Total pickled errors {django_results['Errors'].sum()}, FN {django_results['False Negatives'].sum()}, FP {django_results['False Positives'].sum()}")
print(f"Total  fresh  errors {results['Errors'].sum()}, FN {results['False Negatives'].sum()}, FP {results['False Positives'].sum()}")

Total pickled errors 235, FN 26, FP 209
Total  fresh  errors 105, FN 9, FP 96


In [17]:
django_results

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,65,25,20,5,205,0.92,0.76,0.93,0.84
A2,82,22,22,0,210,0.93,0.79,1.00,0.88
A3,85,3,2,1,209,0.99,0.98,0.99,0.98
B1,50,4,1,3,207,0.98,0.98,0.94,0.96
B2,65,19,19,0,210,0.94,0.77,1.00,0.87
B3,119,31,31,0,210,0.91,0.79,1.00,0.88
B4,79,4,4,0,210,0.99,0.95,1.00,0.98
B5,86,75,75,0,210,0.80,0.53,1.00,0.70
B6,80,52,35,17,193,0.84,0.70,0.82,0.75


In [18]:
results

,Times Occupied,Errors,False Positives,False Negatives,True Negatives,Accuracy,Precision,Recall,F1 Score
A1,65,16,15,1,209,0.94,0.81,0.98,0.89
A2,82,13,13,0,210,0.96,0.86,1.00,0.93
A3,85,2,2,0,210,0.99,0.98,1.00,0.99
B1,50,1,1,0,210,1.00,0.98,1.00,0.99
B2,65,3,3,0,210,0.99,0.96,1.00,0.98
B3,119,0,0,0,210,1.00,1.00,1.00,1.00
B4,79,2,1,1,209,0.99,0.99,0.99,0.99
B5,86,16,16,0,210,0.95,0.84,1.00,0.91
B6,80,36,33,3,207,0.89,0.71,0.96,0.82
